In [1]:
%pylab inline
from ipywidgets import interact_manual

Populating the interactive namespace from numpy and matplotlib


In [2]:
#fname = '../experiment3_1/simulated1_c4_e0.01_k21.dist'
fname = '../data/histograms/e_c4_k21.hist'

In [3]:
import sys
sys.path.append('..')
from covest import *
from models import *
hist_orig, hist = load_hist(fname)

BigFloats are not used!
Precision issues may occur.


In [4]:
model = RepeatsModel(21, 100, hist)

In [5]:
model.compute_loglikelihood(10, 0.05, 0.9, 0.9, 0.9)

-5398132.095137997

In [6]:
import multiprocessing

try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default
    
pool = multiprocessing.Pool(processes=cpus)
print(cpus)

4


In [7]:
from functools import partial
count = 20
base_c = 5
range_c = 3
step_c = range_c/count
base_e = 0.05 * model.err_scale
range_e = 0.05 * model.err_scale
step_e = range_e/count

c_vals = arange(base_c - range_c, base_c + range_c, step_c)
e_vals = arange(base_e - range_e, base_e + range_e, step_e)

def compute_loglikelihood(payload, c, e):
    q1, q2, q = payload
    return model.compute_loglikelihood(c, e, q1, q2, q)


def compute_ll_for_params(q1=0.9, q2=0.9, q=0.9):
    payload = q1, q2, 1
    # f1 = np.array([[compute_loglikelihood(c, e) for e in e_vals] for c in c_vals])
    f1 = array([list(
            pool.map(partial(compute_loglikelihood, payload, c), arange(base_e - range_e, base_e + range_e, step_e))
        ) for c in arange(base_c - range_c, base_c + range_c, step_c)
    ])
    f1[isinf(f1)] = NaN
    return f1


In [8]:
def transform_for_visualisation(a):
    min_val, max_val = nanmin(a), nanmax(a)
    print(max_val)
    def transform(x):
        return log(1 - ((x - min_val) / (max_val - min_val + 1)))
    vtransform = vectorize(transform)
    return vtransform(a)

In [9]:
def draw_plot_for_q(q1=0.9, q2=0.9, q=0.9):
    f1 = compute_ll_for_params(q1, q2, q)
    ax = imshow(        
        transform_for_visualisation(f1), interpolation='none', cmap = 'jet',
        extent=[e_vals[0],e_vals[-1],c_vals[-1],c_vals[0]], aspect='auto',
    )
    colorbar(ax)
    show()

In [10]:
interact_manual(draw_plot_for_q, q1=(0.0, 1.0, .01), q2=(0.0, 1.0, 0.01), q=(0.0, 1.0, 0.01))

Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process

In [ ]:
model.err_scale